In [11]:
!pip install -U boto3
!pip install botocore
!pip install langchain

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/c0/67/4d23a38313d7b37892a6d9c9260809f1a2f5a37feaf6f13da0aa27f57d6d/boto3-1.28.63-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.32.0,>=1.31.63 from https://files.pythonhosted.org/packages/24/0e/39117ec73ea22e700503b3af1dbab270563d6d6ca862cf572899824e4212/botocore-1.31.63-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
Using cached boto3-1.28.63-py3-none-any.whl (135 kB)
Using cached botocore-1.31.63-py3-none-any.whl (11.3 MB)
Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.42
    Uninstalling botocore-1.31.42:
      Successfully uninstalled botocore-1.31.42
  Attempting uninstall: s3transfe

In [96]:
import boto3
import json
from langchain import PromptTemplate
import time
from langchain import PromptTemplate
import json

In [97]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-west-2'
)

In [98]:
# Interact with a large language model (LLM) to generate text 
# based on a prompt.
#
# Arguments:
#   prompt: The text prompt to provide to the LLM.
#   llm_type: The name of the LLM to use, either 'titan' or 'claude'. 
#
# Returns:
#   The text generated by the LLM in response to the prompt.
#   
# This function:
# 1. Prints the llm_type for debugging.
# 2. Formats the prompt into the JSON payload expected by each LLM API.
# 3. Specifies the parameters for text generation like max tokens, temp.
# 4. Calls the Bedrock client to invoke the LLM model API. 
# 5. Parses the response to extract the generated text.
# 6. Returns the generated text string.

def interactWithLLM(prompt, type):

    if type == "titan":
        print("**THE LLM TYPE IS -->" + type)
        print("prompt---->" + prompt)
        # Test for invoke model begins
        parameters = {
            "maxTokenCount": 512,
            "stopSequences": [],
            "temperature": 0,
            "topP": 0.9,
        }
        body = json.dumps({"inputText": prompt, "textGenerationConfig": parameters})
        modelId = "amazon.titan-tg1-large"  # "amazon.titan-tg1-large"
        accept = "application/json"
        contentType = "application/json"
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        response_text_titan = response_body.get("results")[0].get("outputText")

        return response_text_titan

    elif type == "claude":
        print("**THE LLM TYPE IS -->" + type)
        body = json.dumps(
            {
                "prompt": prompt,
                "max_tokens_to_sample": 2048,
                "temperature": 1,
                "top_k": 250,
                "top_p": 0.999,
                "stop_sequences": [],
            }
        )
        modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
        accept = "application/json"
        contentType = "application/json"
        print("prompt---->" + prompt)
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        response_text_claude = response_body.get("completion")

        return response_text_claude

In [99]:
prompt_claude_dynamic = """
Human:  Below are the telemetry data attributes with the range of values which it can have. 
<sample_data>
{telemetry}
</sample_data>
With the sample_data above, generate me 10 lines of sample data and respond in JSON format with the keys {jsonKeys}
Assistant: 
"""

prompt_titan_dynamic = """
Generate me 10 lines of sample data which has below attributes. For each values keep random numbers/values between the values provided. Respond in JSON format with the keys {jsonKeys} . Generate a random set of data points with specified attributes:
{telemetry}

"""

In [100]:
def generateIoTSyntheticData(prompt,llm_type):
    print(prompt)
	#split by comma
    attribute_list = ""
    prompt_split_by_comma = prompt.split(",")
    for str in prompt_split_by_comma:
        split_by_parenthesis = str.split("(")
        print(split_by_parenthesis[0])
        attribute_list = '"' + split_by_parenthesis[0].strip() + '",' + attribute_list
	
    prompt_final = "Return: I don't know"
    
    if llm_type == 'titan':
        prompt_final = prompt_titan_dynamic
    elif llm_type == 'claude':
        prompt_final = prompt_claude_dynamic
    
    prompt_template_for_claud = PromptTemplate.from_template(prompt_final)
    prompt_data_for_claud = prompt_template_for_claud.format(telemetry=prompt,jsonKeys=attribute_list)
    
    response_text = interactWithLLM(prompt_data_for_claud,llm_type)
    
    return response_text

In [101]:
#Enter in the following format with comma seperated as an example seq_number (counter starting with value 1), time_stamp (timestamp with format yyyy-mm-dd hh:mm:ss. Start from 2023-09-14 09:43:12 to 2023-09-15 09:43:12 ), temperature (values between 30 to 50), vibration (values between 100 to 200) , device_status (values between 0 or 1)"

sample_prompt = "seq_number (counter starting with value 1), time_stamp (timestamp with format yyyy-mm-dd hh:mm:ss. Start from 2023-09-14 09:43:12 to 2023-09-15 09:43:12 ), temperature (values between 30 to 50), vibration (values between 100 to 200) , device_status (values between 0 or 1)"

In [102]:
llm_type = "claude"

response = generateIoTSyntheticData(sample_prompt,llm_type)

json_formatted_str = json.dumps(response, indent=2)

print(json_formatted_str)


seq_number (counter starting with value 1), time_stamp (timestamp with format yyyy-mm-dd hh:mm:ss. Start from 2023-09-14 09:43:12 to 2023-09-15 09:43:12 ), temperature (values between 30 to 50), vibration (values between 100 to 200) , device_status (values between 0 or 1)
seq_number 
 time_stamp 
 temperature 
 vibration 
 device_status 
**THE LLM TYPE IS -->claude
prompt---->
Human:  Below are the telemetry data attributes with the range of values which it can have. 
<sample_data>
seq_number (counter starting with value 1), time_stamp (timestamp with format yyyy-mm-dd hh:mm:ss. Start from 2023-09-14 09:43:12 to 2023-09-15 09:43:12 ), temperature (values between 30 to 50), vibration (values between 100 to 200) , device_status (values between 0 or 1)
</sample_data>
With the sample_data above, generate me 10 lines of sample data and respond in JSON format with the keys "device_status","vibration","temperature","time_stamp","seq_number",
Assistant: 

" Here is the generated sample telemet